In [1]:
import sqlalchemy as sa
import pandas as pd
import numpy as np
import re
import pickle

### Def's

In [ ]:
def pull_class_from_db(engine, column, limit):
   
    # Create an SQL command to query data from the 'ke_stage.ba_corpus_1' table
    # for the specified class (column) and limit the number of results to 'Limit'
    sql_command = sa.text(f"""SELECT class, record_id,title, abstract
    FROM ke_stage.ba_corpus_1
    where class = '{column}' 
        and title != '{{}}'
        and LENGTH(title::text) > 6
        and abstract != '{{}}'
        and LENGTH(abstract::text) > 6
    Limit {limit};""")

    # Initialize a list to store the result rows
    result_all = []
    
    # Open a connection to the database and execute the SQL command
    with engine.connect() as connection:
        result = connection.execute(sql_command)
        
        # Iterate over the result set and append each row to the list
        for row in result:
            result_all.append(row)

    return result_all

In [3]:

def pull_data_from_db(engine, limit):
    # Create a dictionary to store the results
    Data = {}
    labels = ['Medizin','ErnÃ¤hrung','Landwirtschaft','Umweltwissenschaften','Rest']

    for i in range(len(labels)): 
       Data[labels[i]] = pull_class_from_db(engine,labels[i],limit)

    return Data

### create connection so SQL DB

In [4]:
# Replace the following values with your actual connection information
DATABASE = 'postgres'
USER = 'postgres'
PASSWORD = 'postgres'
HOST = 'localhost'  
PORT = '5432'

connection_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

# create engine 
engine = sa.create_engine(connection_string)

### create Train-Test dataset

In [5]:
# prepare dataframe all
df_all = pd.DataFrame(columns=['class', 'record_id', 'title', 'abstract'])


In [6]:
# pull train-test data
variables = pull_data_from_db(engine,3000)
for key in variables:
    df_all = pd.concat([df_all, pd.DataFrame(variables[key], columns=['class', 'record_id', 'title', 'abstract'])], ignore_index=True)

In [7]:
df_all.to_pickle('../01_Daten/pkl/df_all_15k-3.pkl')

### create validation dataset

In [8]:
# prepare dataframe val
df_val = pd.DataFrame(columns=['class', 'record_id', 'title', 'abstract'])

In [9]:
# pull validation data
variables = pull_data_from_db(engine,5000)
#reorgnize data
for key in variables:
    df_temp = pd.DataFrame(columns=['class', 'record_id', 'title', 'abstract'])
    df_temp = pd.concat([df_temp, pd.DataFrame(variables[key], columns=['class', 'record_id', 'title', 'abstract'])], ignore_index=True)
    #check for duplikat
    df_temp = df_temp[~df_temp["record_id"].isin(df_all["record_id"])]
    df_temp_1000 = df_temp.sample(n = 1000)
    #create new df
    df_val = pd.concat([df_val, df_temp_1000], ignore_index=True)
    #drop temporary df's
    del df_temp_1000
    del df_temp


In [10]:
df_val.to_pickle('../01_Daten/pkl/df_val_5k-3.pkl')